<a href="https://colab.research.google.com/github/sangramch/EM-BGWO/blob/main/EM-BGWO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import time
import random
import numpy as np
from copy import copy
import tensorflow as tf
from sklearn import metrics
from sklearn import neighbors
from tensorflow.keras import utils
from sklearn import ensemble,tree,svm
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.keras.activations import sigmoid, softmax
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.layers import Dense,Input,Dropout,BatchNormalization

In [2]:
!nvidia-smi

Sun May  2 13:01:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!mkdir dataset
!gdown "https://drive.google.com/uc?id=1zEL8QCuZPq0uF-5MRs7LZUoplDB_Tnt8"
!unzip "/content/Balanced_Morefeatures.zip" -d /content/dataset/

Downloading...
From: https://drive.google.com/uc?id=1zEL8QCuZPq0uF-5MRs7LZUoplDB_Tnt8
To: /content/Balanced_Morefeatures.zip
4.88MB [00:00, 13.3MB/s]
Archive:  /content/Balanced_Morefeatures.zip
   creating: /content/dataset/test/
  inflating: /content/dataset/test/features.npy  
  inflating: /content/dataset/test/labels.npy  
   creating: /content/dataset/train/
  inflating: /content/dataset/train/features.npy  
  inflating: /content/dataset/train/labels.npy  
   creating: /content/dataset/valid/
  inflating: /content/dataset/valid/features.npy  
  inflating: /content/dataset/valid/labels.npy  


In [4]:
!mkdir dataset2
!gdown "https://drive.google.com/uc?id=1PpPGMzkHRzODGCEKBRxPeCFztu0gJ5Wl"
!unzip "/content/Unbalanced_Morefeatures.zip" -d /content/dataset2/

Downloading...
From: https://drive.google.com/uc?id=1PpPGMzkHRzODGCEKBRxPeCFztu0gJ5Wl
To: /content/Unbalanced_Morefeatures.zip
9.31MB [00:00, 20.0MB/s]
Archive:  /content/Unbalanced_Morefeatures.zip
   creating: /content/dataset2/test/
  inflating: /content/dataset2/test/features.npy  
  inflating: /content/dataset2/test/labels.npy  
   creating: /content/dataset2/train/
  inflating: /content/dataset2/train/features.npy  
  inflating: /content/dataset2/train/labels.npy  
   creating: /content/dataset2/valid/
  inflating: /content/dataset2/valid/features.npy  
  inflating: /content/dataset2/valid/labels.npy  


In [5]:
train_features = np.load("/content/dataset/train/features.npy")
train_labels = np.load("/content/dataset/train/labels.npy")

test_features = np.load("/content/dataset/test/features.npy")
test_labels = np.load("/content/dataset/test/labels.npy")

valid_features = np.load("/content/dataset/valid/features.npy")
valid_labels = np.load("/content/dataset/valid/labels.npy")

In [6]:
train_features_u = np.load("/content/dataset2/train/features.npy")
train_labels_u = np.load("/content/dataset2/train/labels.npy")

test_features_u = np.load("/content/dataset2/test/features.npy")
test_labels_u = np.load("/content/dataset2/test/labels.npy")

valid_features_u = np.load("/content/dataset2/valid/features.npy")
valid_labels_u = np.load("/content/dataset2/valid/labels.npy")

In [7]:
total_feats = train_features.shape[1]
mask_f = np.ones(total_feats)

In [8]:
def SvcLearner(features, labels, valid_features, valid_labels, mask):
    mask = np.array(mask)
    inx = np.where(mask==0)
    
    scaler = MinMaxScaler()
    features_masked = scaler.fit_transform(np.delete(features, inx, axis=1))
    valid_features_masked = scaler.transform(np.delete(valid_features, inx, axis=1))
    
    #features_masked = np.delete(features, inx, axis=1)
    #valid_features_masked = np.delete(valid_features, inx, axis=1)

    knn = svm.SVC()

    knn.fit(features_masked, labels)

    predicted_labels = knn.predict(valid_features_masked)

    score = metrics.accuracy_score(valid_labels, predicted_labels)
    return score, [knn, scaler]

In [9]:
def KnnLearner(features, labels, valid_features, valid_labels, mask):
    mask = np.array(mask)
    inx = np.where(mask==0)
    
    scaler = MinMaxScaler()
    features_masked = scaler.fit_transform(np.delete(features, inx, axis=1))
    valid_features_masked = scaler.transform(np.delete(valid_features, inx, axis=1))
    
    #features_masked = np.delete(features, inx, axis=1)
    #valid_features_masked = np.delete(valid_features, inx, axis=1)

    knn = neighbors.KNeighborsClassifier()

    knn.fit(features_masked, labels)

    predicted_labels = knn.predict(valid_features_masked)

    score = metrics.accuracy_score(valid_labels, predicted_labels)
    return score, [knn, scaler]

In [10]:
def DecisionTreeLearner(features, labels, valid_features, valid_labels, mask):
    mask = np.array(mask)
    inx = np.where(mask==0)

    scaler = MinMaxScaler()
    features_masked = scaler.fit_transform(np.delete(features, inx, axis=1))
    valid_features_masked = scaler.transform(np.delete(valid_features, inx, axis=1))

    #features_masked = np.delete(features, inx, axis=1)
    #valid_features_masked = np.delete(valid_features, inx, axis=1)

    knn = tree.DecisionTreeClassifier()
    #knn = ensemble.RandomForestClassifier()

    knn.fit(features_masked, labels)

    predicted_labels = knn.predict(valid_features_masked)

    score = metrics.accuracy_score(valid_labels, predicted_labels)
    return score, [knn, scaler]

In [11]:
class PrintLogs(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        self.epochs = epochs

    def set_params(self, params):
        params['epochs'] = 0

    def on_epoch_begin(self, epoch, logs=None):
        if (epoch+1)%10 == 1:
            print('\tEpoch %d/%d' % (epoch + 1, self.epochs), end='')
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1)%10 == 1:
            print(" Loss: ", logs["loss"], " Acc: ", logs["categorical_accuracy"], " Val Loss: ", logs["val_loss"], " Val Acc: ", logs["val_categorical_accuracy"])

def NeuralNetLearner(features, labels, valid_features, valid_labels, mask):

    mask = np.array(mask)
    inx = np.where(mask==0)

    scaler = MinMaxScaler()
    features_masked = scaler.fit_transform(np.delete(features, inx, axis=1))
    valid_features_masked = scaler.transform(np.delete(valid_features, inx, axis=1))

    _,sub_features,_,sub_labels = train_test_split(features_masked, labels, test_size = 0.025)

    labels = utils.to_categorical(labels)
    valid_labels = utils.to_categorical(valid_labels)

    sub_labels = utils.to_categorical(sub_labels)

    initializer = tf.keras.initializers.GlorotUniform()

    def lr_scheduler(epoch, lr):
        decay_rate = 0.1
        decay_step = [50, 70, 90]
        for step in decay_step:
            if epoch % step == 0 and epoch:
                return lr * decay_rate
        return lr

    inp = Input( shape = (features_masked.shape[1],))
    x = Dense(128, activation = "sigmoid")(inp)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(256, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(512, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(512, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(256, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(256, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(512, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(512, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(256, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(256, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(512, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(1024, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(512, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(256, kernel_initializer = initializer, activation = "relu")(x)
    x = Dense(64, kernel_initializer = initializer, activation = "relu")(x)

    x = Dense(2, activation = "softmax")(x)

    model = Model(inp, x)

    epochs = 100
    callback = [
                 tf.keras.callbacks.LearningRateScheduler(lr_scheduler),
                PrintLogs(epochs)
    ]
    model.compile(loss = "categorical_crossentropy", optimizer = RMSprop(0.0001), metrics = ["categorical_accuracy"])

    model.fit(features_masked, labels, validation_data = (valid_features_masked, valid_labels), epochs = epochs, shuffle = True, verbose=0, callbacks = callback)

    predicted_labels = model.predict(valid_features_masked, batch_size=128)

    score = metrics.accuracy_score(np.argmax(valid_labels, axis=1), np.argmax(predicted_labels,axis=1))
    return score, [model, scaler]

In [12]:
#NeuralNetLearner(train_features, train_labels, valid_features, valid_labels, mask_f)

In [13]:
def Bstep_calc(D_val):
    Cstep = 1 / (1 + math.exp(-10 * (D_val - 0.5)))
    if Cstep >= random.random():
        Bstep = 1
    else:
        Bstep = 0
    
    return Bstep

def Update_wolf(X, Bstep):
    if (X + Bstep) >= 1:
        Y = 1
    else:
        Y = 0
    return Y

def B_GWO(features, labels, valid_features, valid_labels, num_wolves, max_epoch, learner):

    num_features = features.shape[1]

    X_prelim = np.zeros((2*num_wolves, num_features))
    X = list()
    models = [None]*2*num_wolves
    

    for i in range(num_wolves):
        for d in range(num_features):
            if random.random() > 0.5:
                X_prelim[i,d] = 1
    
    for i in range(num_wolves):
        for d in range(num_features):
            if X_prelim[i,d] == 0:
                X_prelim[i+num_wolves,d] = 1
            else:
                X_prelim[i+num_wolves,d] = 0
    
    fit_prelim = np.zeros(2*num_wolves)
    
    fit = np.zeros(num_wolves)

    for i in range(2*num_wolves):
        fit_prelim[i], models[i] = learner(features, labels, valid_features, valid_labels, X_prelim[i])

    idx = np.argsort(fit_prelim)

    for i in range(-1,-num_wolves-1, -1):
        fit[i] = copy(fit_prelim[ idx[i] ])
        X.append( copy(X_prelim[ idx[i] ]) )
    
    X = np.array(X)
    
    X_alpha = copy(X_prelim[ idx[-1] ])
    X_beta = copy(X_prelim[ idx[-2] ])
    X_delta = copy(X_prelim[ idx[-3] ])

    F_alpha = copy(fit_prelim[ idx[-1] ])
    F_beta = copy(fit_prelim[ idx[-2] ])
    F_delta = copy(fit_prelim[ idx[-3] ])

    M_alpha = copy(models[ idx[-1] ])
    M_beta = copy(models[ idx[-2] ])
    M_delta = copy(models[ idx[-3] ])

    curve = list()
    epoch = 1

    while (epoch <= max_epoch):
        #expontiated modified gwo
        a = 4 - (4 * ( (epoch**2) / (max_epoch**2) ))
        #a = 2 - (2 * ( (epoch) / (max_epoch) ))
        for i in range(num_wolves):
            for d in range(num_features):
                C1 = 2 * random.random()
                C2 = 2 * random.random()
                C3 = 2 * random.random()

                D_alpha = abs(C1 * X_alpha[d] - X[i][d])
                D_beta = abs(C2 * X_beta[d] - X[i][d])
                D_delta = abs(C3 * X_delta[d] - X[i][d])

                A1 = (2 * random.random() * a) - a

                Bstep_1 = Bstep_calc(A1 * D_alpha)
                Bstep_2 = Bstep_calc(A1 * D_beta)
                Bstep_3 = Bstep_calc(A1 * D_delta)

                X1 = Update_wolf(X_alpha[d], Bstep_1)
                X2 = Update_wolf(X_beta[d], Bstep_2)
                X3 = Update_wolf(X_delta[d], Bstep_3)

                rand = random.random()

                if rand < 1/3:
                    X[i][d] = X1
                elif rand < 2/3 and rand >= 1/3:
                    X[i][d] = X2
                else:
                    X[i][d] = X3
            
            wolf = copy(X[i])
            wolf_rev = copy(X[i])
            for d in range(num_features):
                if wolf[d] == 0:
                    wolf_rev[d]=1
                else:
                    wolf_rev[d]=0
            
            fitness, mdl = learner(features, labels, valid_features, valid_labels, wolf)
            fitness_rev, mdl_rev = learner(features, labels, valid_features, valid_labels, wolf_rev)

            if fitness>fitness_rev:
                fit[i] = fitness
                models[i] = mdl
            else:
                fit[i] = fitness_rev
                models[i] = mdl_rev
                for d in range(num_features):
                    X[i][d] = wolf_rev[d]
        
        for i in range(num_wolves):
            if fit[i] > F_alpha:
                F_delta = copy(F_beta)
                X_delta = copy(X_beta)
                M_delta = copy(M_beta)

                F_beta = copy(F_alpha)
                X_beta = copy(X_alpha)
                M_beta = copy(M_alpha)

                F_alpha = copy(fit[i])
                X_alpha = copy(X[i])
                M_alpha = copy(models[i])
            
            if fit[i] > F_beta and fit[i] < F_alpha:
                F_delta = copy(F_beta)
                X_delta = copy(X_beta)
                M_delta = copy(M_beta)

                F_beta = copy(fit[i])
                X_beta = copy(X[i])
                M_beta = copy(models[i])
            
            if fit[i] > F_delta and fit[i] < F_alpha and fit[i] < F_beta:
                F_delta = copy(fit[i])
                X_delta = copy(X[i])
                M_delta = copy(models[i])
        
        curve.append(F_alpha)
        print("Best result for BGWO iteration {} = {}, {}, {}".format(epoch, F_alpha, F_beta, F_delta))
        print("Number of features used: {}, {}, {}".format(np.sum(X_alpha), np.sum(X_beta), np.sum(X_delta)))
        epoch += 1
    

    return M_alpha, X_alpha

In [14]:
def printmetric(test_features, test_labels, model, mask, scaler, id=None):
    mask = np.array(mask)
    inx = np.where(mask==0)

    features_masked = scaler.transform(np.delete(test_features, inx, axis=1))

    tm = time.time()
    pred_labels = model.predict(features_masked)
    totaltm = time.time()-tm

    test_labels = 1-test_labels
    pred_labels = 1-pred_labels

    print(id)
    print("-----------------------------------")
    print("Accuracy: ", metrics.accuracy_score(test_labels, pred_labels))
    print("Precision: ", metrics.precision_score(test_labels, pred_labels))
    print("Sensitivity: ", metrics.recall_score(test_labels, pred_labels))

    tn, fp, fn, tp = metrics.confusion_matrix(test_labels, pred_labels).ravel()
    specificity = tn / (tn+fp)

    print("Specificity: ", specificity)
    print("F1 score: ",metrics.f1_score(test_labels,pred_labels))
    print("TPR: ", tp/(tp+fn))
    print("FPR: ", fp/(fp+tn))
    print("Time: ", (totaltm/test_labels.shape[0])*1000000, "microsec")
    print("-----------------------------------")

#Balanced Dataset

In [ ]:
best_svc, mask_svc = B_GWO(train_features, train_labels, valid_features, valid_labels, num_wolves = 5, max_epoch = 40, learner = SvcLearner)
best_svc, scaler_svc = best_svc
score, gen_svc = SvcLearner(train_features, train_labels, valid_features, valid_labels, mask_f)
gen_svc, scaler_svc2 = gen_svc
print("Non BGWO accuracy: ", score)

In [18]:
printmetric(test_features, test_labels, gen_svc, mask_f, scaler_svc2, id="Non BGWO")
printmetric(test_features, test_labels, best_svc, mask_svc, scaler_svc, id="BGWO")

Non BGWO
-----------------------------------
Accuracy:  0.8141592920353983
Precision:  0.7958333333333333
Sensitivity:  0.8451327433628318
Specificity:  0.7831858407079646
F1 score:  0.8197424892703863
TPR:  0.8451327433628318
FPR:  0.2168141592920354
Time:  424.51535705971503 microsec
-----------------------------------
BGWO
-----------------------------------
Accuracy:  0.8119469026548672
Precision:  0.79375
Sensitivity:  0.8429203539823009
Specificity:  0.7809734513274337
F1 score:  0.8175965665236052
TPR:  0.8429203539823009
FPR:  0.21902654867256638
Time:  305.8498939581677 microsec
-----------------------------------


In [ ]:
best_rf, mask_rf = B_GWO(train_features, train_labels, valid_features, valid_labels, num_wolves = 5, max_epoch = 40, learner = DecisionTreeLearner)
best_rf, scaler_rf = best_rf
score, gen_rf = DecisionTreeLearner(train_features, train_labels, valid_features, valid_labels, mask_f)
gen_rf, scaler_rf2 = gen_rf
print("Non BGWO accuracy: ", score)

In [20]:
printmetric(test_features, test_labels, gen_rf, mask_f, scaler_rf2, id="Non BGWO")
printmetric(test_features, test_labels, best_rf, mask_rf, scaler_rf, id="BGWO")

Non BGWO
-----------------------------------
Accuracy:  0.8617256637168141
Precision:  0.8562091503267973
Sensitivity:  0.8694690265486725
Specificity:  0.8539823008849557
F1 score:  0.8627881448957189
TPR:  0.8694690265486725
FPR:  0.14601769911504425
Time:  0.5388154392748807 microsec
-----------------------------------
BGWO
-----------------------------------
Accuracy:  0.8672566371681416
Precision:  0.8721973094170403
Sensitivity:  0.8606194690265486
Specificity:  0.8738938053097345
F1 score:  0.8663697104677061
TPR:  0.8606194690265486
FPR:  0.1261061946902655
Time:  0.6279586690717038 microsec
-----------------------------------


In [ ]:
best_knn, mask_knn = B_GWO(train_features, train_labels, valid_features, valid_labels, num_wolves = 5, max_epoch = 40, learner = KnnLearner)
best_knn, scaler_knn = best_knn
score, gen_knn = KnnLearner(train_features, train_labels, valid_features, valid_labels, mask_f)
gen_knn, scaler_knn2 = gen_knn
print("Non BGWO accuracy: ", score)

In [16]:
printmetric(test_features, test_labels, gen_knn, mask_f, scaler_knn2, id="Non BGWO")
printmetric(test_features, test_labels, best_knn, mask_knn, scaler_knn, id="BGWO")

Non BGWO
-----------------------------------
Accuracy:  0.8871681415929203
Precision:  0.9186602870813397
Sensitivity:  0.8495575221238938
Specificity:  0.9247787610619469
F1 score:  0.8827586206896552
TPR:  0.8495575221238938
FPR:  0.0752212389380531
Time:  456.1255990931418 microsec
-----------------------------------
BGWO
-----------------------------------
Accuracy:  0.8860619469026548
Precision:  0.9086651053864169
Sensitivity:  0.8584070796460177
Specificity:  0.9137168141592921
F1 score:  0.882821387940842
TPR:  0.8584070796460177
FPR:  0.08628318584070796
Time:  264.8838325939347 microsec
-----------------------------------


In [ ]:
mask_f = np.ones((87))
best_net, mask_net = B_GWO(train_features, train_labels, valid_features, valid_labels, num_wolves = 5, max_epoch = 40, learner = NeuralNetLearner)
best_net, scaler_net = best_net
score, gen_net = NeuralNetLearner(train_features, train_labels, valid_features, valid_labels, mask_f)
gen_net, scaler_net2 = gen_net
print("Non BGWO accuracy: ", score)

In [ ]:
printmetric(test_features, test_labels, gen_net, mask_f, scaler_net2, id="Non BGWO")
printmetric(test_features, test_labels, best_net, mask_net, scaler_net, id="BGWO")

#Unbalanced Dataset

In [ ]:
best_svc, mask_svc = B_GWO(train_features_u, train_labels_u, valid_features_u, valid_labels_u, num_wolves = 5, max_epoch = 40, learner = SvcLearner)
best_svc, scaler_svc = best_svc
score, gen_svc = SvcLearner(train_features_u, train_labels_u, valid_features_u, valid_labels_u, mask_f)
gen_svc, scaler_svc2 = gen_svc
print("Non BGWO accuracy: ", score)

Best result for BGWO iteration 1 = 0.8719582850521437, 0.8690614136732329, 0.8684820393974507
Number of features used: 60.0, 51.0, 51.0
Best result for BGWO iteration 2 = 0.8736964078794901, 0.8719582850521437, 0.8713789107763615
Number of features used: 53.0, 60.0, 67.0
Best result for BGWO iteration 3 = 0.8736964078794901, 0.8719582850521437, 0.8713789107763615
Number of features used: 53.0, 60.0, 67.0
Best result for BGWO iteration 4 = 0.8754345307068366, 0.8736964078794901, 0.8725376593279258
Number of features used: 60.0, 53.0, 64.0
Best result for BGWO iteration 5 = 0.8754345307068366, 0.8736964078794901, 0.873117033603708
Number of features used: 60.0, 53.0, 63.0
Best result for BGWO iteration 6 = 0.8754345307068366, 0.8748551564310545, 0.8736964078794901
Number of features used: 60.0, 65.0, 53.0
Best result for BGWO iteration 7 = 0.8754345307068366, 0.8748551564310545, 0.8736964078794901
Number of features used: 60.0, 65.0, 53.0
Best result for BGWO iteration 8 = 0.876593279258

In [ ]:
printmetric(test_features_u, test_labels_u, gen_svc, mask_f, scaler_svc2, id="Non BGWO")
printmetric(test_features_u, test_labels_u, best_svc, mask_svc, scaler_svc, id="BGWO")

In [ ]:
best_rf, mask_rf = B_GWO(train_features_u, train_labels_u, valid_features_u, valid_labels_u, num_wolves = 5, max_epoch = 40, learner = DecisionTreeLearner)
best_rf, scaler_rf = best_rf
score, gen_rf = DecisionTreeLearner(train_features_u, train_labels_u, valid_features_u, valid_labels_u, mask_f)
gen_rf, scaler_rf2 = gen_rf
print("Non BGWO accuracy: ", score)

In [ ]:
printmetric(test_features_u, test_labels_u, gen_rf, mask_f, scaler_rf2, id="Non-BGWO")
printmetric(test_features_u, test_labels_u, best_rf, mask_rf, scaler_rf, id="BGWO")

In [ ]:
best_knn, mask_knn = B_GWO(train_features_u, train_labels_u, valid_features_u, valid_labels_u, num_wolves = 5, max_epoch = 40, learner = KnnLearner)
best_knn, scaler_knn = best_knn
score, gen_knn = KnnLearner(train_features_u, train_labels_u, valid_features_u, valid_labels_u, mask_f)
gen_knn, scaler_knn2 = gen_knn
print("Non BGWO accuracy: ", score)

In [ ]:
printmetric(test_features_u, test_labels_u, gen_knn, mask_f, scaler_knn2, id="Non BGWO")
printmetric(test_features_u, test_labels_u, best_knn, mask_knn, scaler_knn, id="BGWO")

In [ ]:
mask_f = np.ones((87))
best_net, mask_net = B_GWO(train_features_u, train_labels_u, valid_features_u, valid_labels_u, num_wolves = 5, max_epoch = 40, learner = NeuralNetLearner)
best_net, scaler_net = best_net
score, gen_net = NeuralNetLearner(train_features_u, train_labels_u, valid_features_u, valid_labels_u, mask_f)
gen_net, scaler_net2 = gen_net
print("Non BGWO accuracy: ", score)

In [ ]:
printmetric(test_features_u, test_labels_u, gen_net, mask_f, scaler_net2, id="Non BGWO")
printmetric(test_features_u, test_labels_u, best_net, mask_net, scaler_net, id="BGWO")